## graphcol_3
* graph loader/converter for dimacs `col` format: `loadcol`
* the data is from [Michael Trick's page](https://mat.gsia.cmu.edu/COLOR/instances.html)
* dimacs is an old/simple format described [here](https://mat.gsia.cmu.edu/COLOR/general/ccformat.ps)

In [10]:
include("../shared/tomdcode.jl")
_JLAB_=true

true

In [11]:
md"""### The `grapcol_3` module
* the `tolg` parameter for converting the data into the default `lg` format
* for the instances where the optimum is known, the value is built into the file, and if `toopt` is true a separate file will be created containing it.
"""|>display
tomdcode("src/graphcol_3.jl")

### The `grapcol_3` module

  * the `tolg` parameter for converting the data into the default `lg` format
  * for the instances where the optimum is known, the value is built into the file, and if `toopt` is true a separate file will be created containing it.


```julia
module graphcol_3
  # .col extension is a must
  function loadcol(gfile::String; tolg = false, toopt = false)
    _e(msg) = error("loadcol: $(msg)")

    !isfile(gfile) && _e("no such file")
    sfile = split(gfile, '.')
    (sfile[end] != "col") && _e("wrong extension")
    gstring = split(read(gfile, String), '\n', keepempty = false)

    num_colors = -1
    E = []
    nV, nE = -1, -1
    for line in gstring
      sline = split(line, keepempty = false)
      (sline[1] == "c") && continue

      if sline[1] == "p" # only the last counts, but must precede the first 'p' line, bcos it  is used in a sanity check
        nV, nE = parse.(Int, sline[3:end])
        continue
      end
      if sline[1] == "e"
        a, b = parse.(Int, split(line)[2:end])
        if a < 1 || a > nV || b < 1 || b > nV
          _e("vertex is out of range")
        end
        push!(E, (a, b))
        continue
      end
      if sline[1] == "num_colors"
        num_colors = parse(Int, sline[2])
        continue
      end
    end
    if nV < 0 || nE < 0 || length(E) != nE
      _e("wrong data")
    end
    G = Graph()
    add_vertices!(G, nV)
    for (a, b) in E
      add_edge!(G, a, b)
    end
    if tolg == true
      sfile = join(sfile[1:end-1], '.')
      savegraph("$(sfile).lg", G)
      printstyled(stderr, "saved $(sfile).lg\n"; color = :green)
    end
    if num_colors > 0 && toopt == true
      open("$(sfile).opt", "w") do f
        println(f, num_colors)
      end
      printstyled(stderr, "saved $(sfile).opt\n"; color = :yellow)
    end
    G
  end
  export loadcol

end


```


In [12]:
md"""### The client code
"""|>display
tomdcode("main.jl")

### The client code


```julia
include("todev.jl"); todev(["../graphcol_2/src"])
using graphcol_2: graphcol_bt
using Graphs

#### convert the col's to lg's
printstyled("convert the col's to lg's in a directory\n",color=:light_white)
for f in readdir("../data/col-instances/"; join = true)
  sf = split(f, '.')
  if sf[end] == "col"
    jf = join(sf[1:end-1], '.')
    isfile("$(jf).lg") && isfile("$(jf).opt") && continue
    loadcol(f; tolg = true, toopt = true)
  end
end

# test 
printstyled("""
\nperform some tests w/ the newly converted data+graphcol_bt
note, that we know optimum and it will be used
""",color=:light_blue)
G = loadgraph("../data/col-instances/queen5_5.lg")
opt = parse(Int, read("../data/col-instances/queen5_5.opt", String))
printstyled("\ndata: queen5_5.lg, opt=$(opt)\n", color=:light_yellow)
printstyled("\ngraphcol_bt+coloring w/ $(opt-1) colors\n", color=:green)
@time graphcol_bt(G, opt - 1) |> println
printstyled("\ngraphcol_bt+coloring w/ $(opt) colors\n", color=:green)
@time graphcol_bt(G, opt) |> println
reps=33
printstyled("\ngreedy_color, reps=$(reps)\n", color=:green)
@time greedy_color(G, reps = reps) |> println

```


In [13]:
md"""### The output
"""|>display
include("main.jl")

### The output


convert the col's to lg's in a directory

perform some tests w/ the newly converted data+graphcol_bt
note, that we know optimum and it will be used

data: queen5_5.lg, opt=5

graphcol_bt+coloring w/ 4 colors
(num_colors = -1, colors = nothing)
  0.000133 seconds (198 allocations: 16.172 KiB)

graphcol_bt+coloring w/ 5 colors
(num_colors = 5, colors = [1, 2, 3, 4, 5, 3, 4, 5, 1, 2, 5, 1, 2, 3, 4, 2, 3, 4, 5, 1, 4, 5, 1, 2, 3])
  0.000598 seconds (1.03 k allocations: 36.594 KiB)

greedy_color, reps=33
Graphs.Coloring{Int64}(5, [4, 3, 5, 2, 1, 5, 2, 1, 4, 3, 1, 4, 3, 5, 2, 3, 5, 2, 1, 4, 2, 1, 4, 3, 5])
  0.001138 seconds (1.99 k allocations: 118.289 KiB)
